In [4]:
import numpy as np 
import pandas as pd 

In [6]:
from sklearn.impute import SimpleImputer # missing values 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [7]:
df = pd.read_csv('covid_toy.csv')

In [8]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [9]:
# check missing values
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [13]:
# all categorical variable counts  
df['gender'].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [14]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [15]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [24]:
# split the data set 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns =['has_covid']),df['has_covid'],
                                                test_size = 0.2)

In [25]:
X_train

,age,gender,fever,cough,city
45,72,Male,99.0,Mild,Bangalore
67,65,Male,99.0,Mild,Bangalore
5,84,Female,NaN,Mild,Bangalore
27,33,Female,102.0,Strong,Delhi
29,34,Female,NaN,Strong,Mumbai
...,...,...,...,...,...
41,82,Male,NaN,Mild,Kolkata
81,65,Male,99.0,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
75,5,Male,102.0,Mild,Kolkata


In [26]:
y_test

1     Yes
88     No
49     No
96    Yes
2      No
12     No
7     Yes
78    Yes
56     No
59    Yes
31     No
21    Yes
52    Yes
3      No
71     No
97     No
53    Yes
73    Yes
58    Yes
13    Yes
Name: has_covid, dtype: object

# 1.Aam Zindagi

In [31]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [35]:
# Ordinal encoding -> cough 
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data 
X_test_cough = oe.fit_transform(X_test[['cough']])
X_test_cough.shape

(20, 1)

In [41]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [46]:
# Extracting age 
X_train_age = X_train.drop(columns = ['gender','fever','cough','city']).values

# also test data 
X_test_age = X_test.drop(columns = ['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [53]:

# Convert sparse matrices to dense arrays
X_train_gender_city_dense = X_train_gender_city.toarray()
X_test_gender_city_dense = X_test_gender_city.toarray()

# Concatenate all arrays
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city_dense, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city_dense, X_test_cough), axis=1)

X_train_transformed.shape

(80, 7)

In [54]:
from sklearn.compose import ColumnTransformer

In [74]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')


In [75]:
# fitted train data 

transformer.fit_transform(X_train).shape


(80, 7)

In [76]:
# fitted test data
transformer.transform(X_test).shape


(20, 7)